In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import time


# 1. Roll Call Votes 115th Congress - 1st Session (2017)

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("- - icognito")

driver = webdriver.Chrome(chrome_options = chrome_options)
driver.get("https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.htm")


In [3]:
Senate115 = driver.page_source
Senate115_soup = BeautifulSoup(Senate115, "html.parser")
# Senate115_soup


In [4]:
Senate115_soup_table = Senate115_soup.find_all("table")
# Senate115_soup_table


In [5]:
Senate115_soup_table_0 = Senate115_soup_table[0]#.text
# Senate115_soup_table_0



In [6]:
Senate115_soup_table_0_tr = Senate115_soup_table_0.find_all("tr")
# Senate115_soup_table_0_tr


In [7]:
vote_link_list = []
issue_link_list = []

for element in Senate115_soup_table_0_tr[1:]: # `[1:]` to exclude the column names
    element_td = element.find_all("td")

#     if  not "PN"          in element_td[3].text \
#     and not "Treaty Doc." in element_td[3].text:
    element_td_0_a = element_td[0].find_all('a')
    vote_link_list.append("https://www.senate.gov"+element_td_0_a[0]['href']) # Add "https://www.senate.gov"
    
    element_td_3_a = element_td[3].find_all('a')
    issue_link_list.append(element_td_3_a[0]['href'])

# vote_link_list
issue_link_list_without_duplicates = list(set(issue_link_list))

In [8]:
# 'https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=115&session=1&vote=00187'
# "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00187.xml"
# "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00186.xml"

vote_xml_link_list = []

for vote_link in vote_link_list:
    vote_xml_link = vote_link.replace("/roll_call_lists/roll_call_vote_cfm.cfm?congress=115&session=1&vote=", 
                                      "/roll_call_votes/vote1151/vote_115_1_")
    vote_xml_link = vote_xml_link + ".xml"
    
    vote_xml_link_list.append(vote_xml_link)

vote_xml_link_list_without_duplicates = list(set(vote_xml_link_list))

In [9]:
vote_xml_link_issue_link_list = list(zip(vote_xml_link_list, issue_link_list))

# 2. individual vote pages -> xml pages

In [10]:
def scrape_votes(start_index, end_index):
    vote_xml_list = []
    
    if len(vote_xml_link_list_without_duplicates) < end_index:
        end_index = len(vote_xml_link_list_without_duplicates)
    
    for vote_xml_link in vote_xml_link_list_without_duplicates[start_index:end_index]:
        driver.get(vote_xml_link)
        time.sleep(2)
        vote_xml_list.append(driver.page_source)

    return vote_xml_list

In [11]:
# len(vote_xml_link_list_without_duplicates) # 187
vote_xml_list_0_50 = scrape_votes(0, 50)

In [12]:
vote_xml_list_50_100 = scrape_votes(50, 100)

In [13]:
vote_xml_list_100_187 = scrape_votes(100, 187)

In [15]:
vote_xml_list = vote_xml_list_0_50 + vote_xml_list_50_100 + vote_xml_list_100_187
vote_xml_link_to_xml = dict(zip(vote_xml_link_list_without_duplicates, vote_xml_list))


# 3. individual issue pages

In [23]:
def scrape_issues(start_index, end_index):
    issue_html_list = []
    
    if len(issue_link_list_without_duplicates) < end_index:
        end_index = len(issue_link_list_without_duplicates)
    
    for issue_link in issue_link_list_without_duplicates[start_index:end_index]:
        driver.get(issue_link)
        time.sleep(2)
        issue_html_list.append(driver.page_source)

    return issue_html_list

In [24]:
# len(issue_link_list_without_duplicates) # 81
issue_html_list_0_50 = scrape_issues(0, 50)

In [25]:
issue_html_list_50_81 = scrape_issues(50, 81)

In [26]:
issue_html_list = issue_html_list_0_50 + issue_html_list_50_81
issue_link_to_html = dict(zip(issue_link_list_without_duplicates, issue_html_list))


# 4. save to files, just in case

In [27]:
import json

In [28]:
with open('vote_xml_link_to_xml.py', 'w') as f:
    json.dump(vote_xml_link_to_xml, f, default=repr, indent=4)

In [29]:
with open('issue_link_to_html.py', 'w') as f:
    json.dump(issue_link_to_html, f, default=repr, indent=4)

# 5. get just the info I want from vote_xml

In [30]:
for vote_xml_link in vote_xml_link_to_xml:
    print(vote_xml_link)
    break

# <member>
#     <member_full>Alexander (R-TN)</member_full>
#     <last_name>Alexander</last_name>
#     <first_name>Lamar</first_name>
#     <party>R</party>
#     <state>TN</state>
#     <vote_cast>Yea</vote_cast>
#     <lis_member_id>S289</lis_member_id>
# </member>


#         for member in member_list:
#             vote_info = {
#                 "Senator"       : member.find('first_name').text + " " + member.find('last_name').text,
#                 "Party"         : member.find('party').text,
#                 "State"         : member.find('state').text,
#                 "Question"      : page_doc.find('vote_document_text').text,
#                 "Vote number"   : page_doc.find('vote_number').text,
#                 "Vote cast"     : member.find('vote_cast').text,
#                 "Sponsor Party" : sponsor_party.groups(1)[0],
#             }
#             vote_data.append(vote_info)

https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00086.xml


# 6. get just the info I want from issue_html

In [38]:
import re

In [55]:
issue_link_to_details = {}

for issue_link in issue_link_to_html:
    issue_details = {}
    print(issue_link)
    
    issue_html = issue_link_to_html[issue_link]
    issue_soup = BeautifulSoup(issue_html, 'html.parser')
    
    issue_soup_tr = issue_soup.find_all('tr')
    for issue_soup_tr_i in issue_soup_tr:
        if "Sponsor" in issue_soup_tr_i.text:
            sponsor_td_text = issue_soup_tr_i.find_next('td').text
            sponsor_party_matchedobject = re.search(r'\[(.)-', sponsor_td_text)
            
            issue_details["sponsor_td_text"] = sponsor_td_text
            issue_details["sponsor_party"]   = sponsor_party_matchedobject.groups(1)[0]
            print(issue_details)
            break
    
    issue_soup_h2 = issue_soup.find_all('h2')
    for issue_soup_h2_i in issue_soup_h2:
        if "Description" in issue_soup_h2_i.text \
        or "Formal Title" in issue_soup_h2_i.text \
        or "Nomination" in issue_soup_h2_i.text:
            description_ul_text = issue_soup_h2_i.find_next('ul').text
            
            issue_details["description_ul_text"] = description_ul_text.strip()
            print(issue_details)
            break
    
    if not issue_details:
        print("NOTHING???")
    
    issue_details['label_text'] = issue_soup.find('h2', 'label_text').text
    issue_link_to_details[issue_link] = issue_details

https://www.congress.gov/nomination/115th-congress/52
{'description_ul_text': "Terry Branstad, of Iowa, to be Ambassador Extraordinary and Plenipotentiary of the United States of America to the People's Republic of China."}
http://www.congress.gov/bill/115th-congress/house-joint-resolution/44
{'sponsor_td_text': 'Rep. Cheney, Liz [R-WY-At Large] (Introduced 01/30/2017)', 'sponsor_party': 'R'}
http://www.congress.gov/bill/115th-congress/house-joint-resolution/69
{'sponsor_td_text': 'Rep. Young, Don [R-AK-At Large] (Introduced 02/07/2017)', 'sponsor_party': 'R'}
https://www.congress.gov/nomination/115th-congress/681
{'description_ul_text': 'Marvin Kaplan, of Kansas, to be a Member of the National Labor Relations Board for the term of five years expiring August 27, 2020, vice Harry I. Johnson III, resigned.'}
https://www.congress.gov/nomination/115th-congress/365
{'description_ul_text': 'David Bernhardt, of Virginia, to be Deputy Secretary of the Interior, vice Michael L. Connor.'}
https:

{'description_ul_text': 'Linda E. McMahon, of Connecticut, to be Administrator of the Small Business Administration, vice Maria Contreras-Sweet, resigned.'}
https://www.congress.gov/nomination/115th-congress/348
{'description_ul_text': 'Kenneth P. Rapuano, of Virginia, to be an Assistant Secretary of Defense, vice Eric Rosenbach, resigned.'}
http://www.congress.gov/bill/115th-congress/senate-joint-resolution/42
{'sponsor_td_text': 'Sen. Paul, Rand [R-KY] (Introduced 05/25/2017)', 'sponsor_party': 'R'}
http://www.congress.gov/bill/115th-congress/senate-resolution/176
{'sponsor_td_text': 'Sen. McConnell, Mitch [R-KY] (Introduced 05/24/2017)', 'sponsor_party': 'R'}
https://www.congress.gov/nomination/115th-congress/101
{'description_ul_text': 'Heather Wilson, of South Dakota, to be Secretary of the Air Force, vice Deborah Lee James.'}
https://www.congress.gov/nomination/115th-congress/49
{'description_ul_text': 'Seema Verma, of Indiana, to be Administrator of the Centers for Medicare and 

In [56]:
issue_link_to_details

{'http://www.congress.gov/bill/115th-congress/house-bill/1628': {'label_text': 'Bill',
  'sponsor_party': 'R',
  'sponsor_td_text': 'Rep. Black, Diane [R-TN-6] (Introduced 03/20/2017)'},
 'http://www.congress.gov/bill/115th-congress/house-bill/2430': {'label_text': 'Bill',
  'sponsor_party': 'R',
  'sponsor_td_text': 'Rep. Walden, Greg [R-OR-2] (Introduced 05/16/2017)'},
 'http://www.congress.gov/bill/115th-congress/house-bill/244': {'label_text': 'Law',
  'sponsor_party': 'R',
  'sponsor_td_text': 'Rep. Cook, Paul [R-CA-8] (Introduced 01/04/2017)'},
 'http://www.congress.gov/bill/115th-congress/house-bill/3364': {'label_text': 'Law',
  'sponsor_party': 'R',
  'sponsor_td_text': 'Rep. Royce, Edward R. [R-CA-39] (Introduced 07/24/2017)'},
 'http://www.congress.gov/bill/115th-congress/house-bill/72': {'label_text': 'Law',
  'sponsor_party': 'R',
  'sponsor_td_text': 'Rep. Carter, Earl L. "Buddy" [R-GA-1] (Introduced 01/03/2017)'},
 'http://www.congress.gov/bill/115th-congress/house-joint